# Sieci rekurencyjne (RNN)

## Inicjalizacja wag sieci

In [1]:
import numpy as np
np.random.seed(0)
def generate_random_matrix(rows, cols): # funkcja generująca macierz o wartościach <-1, 1> o zadanych rozmiarach
    x = np.random.rand(rows, cols)
    y = np.random.rand(rows, cols)
    for i in range(rows):
        for j in range(cols):
            if (y[i][j] > 0.5):
                x[i][j] *= -1
    return x

print(generate_random_matrix(4, 3))

[[-0.5488135  -0.71518937  0.60276338]
 [ 0.54488318  0.4236548  -0.64589411]
 [-0.43758721 -0.891773   -0.96366276]
 [-0.38344152  0.79172504 -0.52889492]]


## Wyznaczenie aktywacji warstwy ukrytej (pamięci) i wyjściowej

W przypadku sieci rekurencyjnych pojawia się połączenie rekursywne z historycznymi danymi. 
Dla prostej rekurencyjnej sieci neuronowej z jedną warstwą ukrytą, obliczenia będą wyglądać następująco:

$\vec{a_{h1(t)}} = sigmoid(U\vec{x} + W\vec{a_{h1(t-1)}} + \vec{b_1})$ - aktywacja f. sigmoid jest tylko przykładem,  można użyć innej aktywacji (tanh, relu, etc)

$\vec{a_{o(t)}} = softmax(V\vec{a_{h1(t)}} + \vec{b_2})$ - softmax jest przykładem f. aktywacji, ma sens w problemie klasyfikacji przy > 2 etykietach, można również użyć np. sigmoid (jeśli jeden neuron na wyjściu)

<br/>

gdzie $\vec{a_{h1(t)}}$ to wartość wektora reprezentującego aktywację warstwy ukrytej w aktualnym kroku,

$\vec{a_{h1(t-1)}}$ to wartość aktywacji warstwy ukrytej w poprzednim kroku,

$\vec{a_{o(t)}}$ to rezultat wygenerowany przez całą sieć neuronową (aktywacja na warstwie "output")

In [2]:
def sigmoid(x): 
    output = 1 / (1 + np.exp(-x))
    return output

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

U = np.array([[-3., 2., ], [4., 2., ], [1., -5.,]])
W = np.array([[1.25, 1.3, 1.5], [2.01, 3.4, -2.4], [1.08, -.3, 0.1]])
V = np.array([[-0.2,0.81, -0.2], [0.12, 0.42, 0.21], [0.1, 0.32, 0.01]])

x = np.array([0.5, 0.21])
prev_hidden = np.array([0.1, 0.32, 0.01])

def get_hidden_state_activation(U, W, x, prev_hidden): # funkcja obliczająca aktualną wartość wektora w warstwie ukrytej (pamięć o sekwencji)
    vector = U.dot(x) + W.dot(prev_hidden)
    output = sigmoid(vector)
    return output

def get_network_output(V, current_hidden): # funkcja obliczająca wyjście sieci dla aktualnie obliczonych wartości w warstwie ukrytej
    vector = V.dot(current_hidden)
    output = softmax(vector)
    return output


hidden_state = get_hidden_state_activation(U, W, x, prev_hidden)
print(hidden_state)
print(get_network_output(V, hidden_state))

[0.37191738 0.97551727 0.36888573]
[0.37811473 0.33866203 0.28322324]


## Przykładowa implementacja sieci neuronowej dodająca do siebie dwie liczby binarne (many-to-many)
Dla uproszczenia liczby wejściowe i wyjściowe są tej samej długości

In [3]:
import copy
import numpy as np
np.random.seed(0)

def sigmoid(x): 
    output = 1 / (1 + np.exp(-x))
    return output

def sigmoid_output_to_derivative(output):
    return output * (1 - output)

def load_dataset(path): #wczytanie danych wejściowych w formacie liczba1_binarnie,liczba2_binarnie,ich_suma_binarnie
    X = []  # lista par składników
    Y = []  # lista sum składników
    with open(path, 'r') as f:
        for line in f:
            if len(line.strip()) == 0:
                continue
            input1_bin, input2_bin, sum_bin = line.strip().split(",")
            
            input1_bin = np.array([int(i) for i in input1_bin]) # przekształcenie liczb na wektory
            input2_bin = np.array([int(i) for i in input2_bin]) # przekształcenie liczb na wektory
            sum_bin = np.array([int(i) for i in sum_bin])
            
            X.append((input1_bin, input2_bin)) # zapisanie par składników jako wejścia
            Y.append(sum_bin)                  # zapisanie sumy jako oczekiwany rezultat
    return X, Y


train_X, train_Y = load_dataset('dataset.csv')

alpha = 0.1      # stała uczenia (learning_rate) - jak duże zmiany robić w uczeniu
input_dim = 2    # liczba cech (liczb) na wejściu sieci
hidden_dim = 16  # rozmiar warstwy ukrytej (rozmiar wektora z pamięcią)
output_dim = 1   # liczba wartości na wyjściu 

U = generate_random_matrix(input_dim, hidden_dim)   # inicjalizacja macierzy między wejściem a warstwą ukrytą
V = generate_random_matrix(hidden_dim, output_dim)  # inicjalizacja macierzy między warstwą ukrytą a wyjściową
W = generate_random_matrix(hidden_dim, hidden_dim)  # inicjalizacja macierzy między poprzednim stanem warstwy ukrytej a aktualnym

U_update = np.zeros_like(U) # macierz poprawek, które aplikowane są w procesie uczenia, aby wyznaczyć coraz lepsze wartości wag
V_update = np.zeros_like(V) # macierz poprawek, które aplikowane są w procesie uczenia, aby wyznaczyć coraz lepsze wartości wag
W_update = np.zeros_like(W) # macierz poprawek, które aplikowane są w procesie uczenia, aby wyznaczyć coraz lepsze wartości wag

for j in range(len(train_Y)):    # iteracja po wszystkich przykładach uczących
    added_one_seq = train_X[j][0]   # pierwszy składnik do sumowania w postaci binarnej, np. 00001
    added_two_seq = train_X[j][1]   # drugi składnik do sumowania w postaci binarnej, np. 00010
    expected_sum_seq = train_Y[j]   # oczekiwany wynik sumowania obu składników (dla przykładu wyżej: 00011)

    predicted_sum_seq = np.zeros_like(expected_sum_seq) # pusty wektor, który wypełniany będzie wartościami 0 lub 1 tworząc predykcję

    overallError = 0                # różnica przewidywań od wartości oczekiwanej  

    output_l_deltas = list()
    hidden_l_values = list()
    hidden_l_values.append(np.zeros(hidden_dim))

    # iteracja po postaci binarnej bit po bicie, od najmniej znaczącego (od prawej do lewej)
    for position in range(len(added_one_seq) - 1, -1, -1):

        # jako wejście sieci w aktualnym kroku brana jest para bitów na pozycji [position] (2 liczby)
        X = np.array([
            [added_one_seq[position], added_two_seq[position]]
        ])
        # jako oczekiwane wyjście sieci w aktualnym kroku brany jest bit na pozycji [position] (odpowiedź budowana jest jako sekwencja, znak po znaku)
        y = np.array([[expected_sum_seq[position]]]).T

        # obliczenie wartość warstwy ukrytej
        hidden_l = get_hidden_state_activation(X, hidden_l_values[-1], U, W)
        
        # pobranie wygenerowanego wyjścia sieci neuronowej (sigmoida, a nie softmax, gdyż jest tylko jedno wyjście binarne)
        predicted_char = sigmoid(np.dot(hidden_l, V))

        # sprawdzenie błędu
        output_l_error = predicted_char - y

        output_l_deltas.append(
            (output_l_error) * sigmoid_output_to_derivative(predicted_char)   # loss * grad out
        )
        overallError += np.abs(output_l_error[0]) # zwiększenie całościowego błędu o błąd z aktualnego kroku

        predicted_sum_seq[position] = np.round(predicted_char[0][0]) # zapisanie przewidywania na odpowiedniej pozycji, zaokrąglając (predicted_char to wartość rzeczywista między 0 a 1, zawiera prawdopodobieństwo tego, że liczba powinna być 1-nką) 

        hidden_l_values.append(copy.deepcopy(hidden_l)) # zapisanie wartości aktywacji warstwy ukrytej, aby można było ją użyć w kolejnym kroku

    future_hidden_l_delta = np.zeros(hidden_dim)        
    
    # propagacja wsteczna 
    for position in range(len(added_one_seq)):              
        X = np.array([
            [added_one_seq[position], added_two_seq[position]]
        ])  # weź parę liczb od lewej do prawej
        hidden_l = hidden_l_values[-position - 1] 
        prev_hidden_l = hidden_l_values[-position - 2]

        # błąd na warstwie wyjściowej
        output_l_delta = output_l_deltas[-position - 1]
        # błąd na warstwie ukrytej
        hidden_l_delta = (future_hidden_l_delta.dot(W.T) + output_l_delta.dot(V.T)) * sigmoid_output_to_derivative(hidden_l)

        # aktualizacja macierzy poprawek względem błędu w aktualnym kroku (na aktualnej pozycji w sekwencji)
        V_update += np.atleast_2d(hidden_l).T.dot(output_l_delta)
        W_update += np.atleast_2d(prev_hidden_l).T.dot(hidden_l_delta)
        U_update += X.T.dot(hidden_l_delta)

        future_hidden_l_delta = hidden_l_delta

    U -= U_update * alpha  # spadek wag między wejściem a w. ukrytą
    V -= V_update * alpha  # spadek wag między w. ukrytą a wyjściem
    W -= W_update * alpha  # spadek wag między poprzednią a teraźniejszą w. ukrytą

    U_update *= 0  # zerowanie macierz
    V_update *= 0  # zerowanie macierz
    W_update *= 0  # zerowanie macierz

    if(j % 5000 == 0):
        print("Błąd przykładu:" + str(overallError))
        print("Przewidziana sekwencja:" + str(predicted_sum_seq))
        print("Oczekiwana sekwencja:  " + str(expected_sum_seq))
        print("------------")

Błąd przykładu:[4.02600273]
Przewidziana sekwencja:[1 1 1 0 0 1 0 0]
Oczekiwana sekwencja:  [0 1 0 0 0 1 0 1]
------------
Błąd przykładu:[1.50900826]
Przewidziana sekwencja:[1 1 0 0 0 0 1 0]
Oczekiwana sekwencja:  [1 1 0 0 0 0 1 0]
------------
Błąd przykładu:[0.32491466]
Przewidziana sekwencja:[0 1 0 1 1 1 1 1]
Oczekiwana sekwencja:  [0 1 0 1 1 1 1 1]
------------
Błąd przykładu:[0.16574001]
Przewidziana sekwencja:[0 1 1 0 1 0 0 1]
Oczekiwana sekwencja:  [0 1 1 0 1 0 0 1]
------------
Błąd przykładu:[0.21679244]
Przewidziana sekwencja:[1 0 0 1 1 0 1 1]
Oczekiwana sekwencja:  [1 0 0 1 1 0 1 1]
------------
Błąd przykładu:[0.15873883]
Przewidziana sekwencja:[0 1 0 0 0 1 0 1]
Oczekiwana sekwencja:  [0 1 0 0 0 1 0 1]
------------
Błąd przykładu:[0.15120312]
Przewidziana sekwencja:[1 0 1 0 1 0 1 0]
Oczekiwana sekwencja:  [1 0 1 0 1 0 1 0]
------------
Błąd przykładu:[0.15645891]
Przewidziana sekwencja:[1 0 1 0 1 1 1 1]
Oczekiwana sekwencja:  [1 0 1 0 1 1 1 1]
------------
Błąd przykładu:[